In [85]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import numpy as np
import keras
import tensorflow
from keras.optimizers import SGD
import collections

import sys
sys.path.append("..")
from utils import *
from funcs import *

from fcnn_att import model_fcnn
from DCASE_training_functions import *

from tensorflow import ConfigProto
from tensorflow import InteractiveSession

%load_ext autoreload
%autoreload 2

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

base_path = '/work/user_data/jtaylor/data/acoustic_scene_classification'
train_csv = f'{base_path}/data/CochlScene/train_shuffled_20k.tsv'
val_csv = f'{base_path}/data/CochlScene/val_shuffled_5k.tsv'

# train_csv = f'{base_path}/data/CochlScene/train_shuffled_tiny.tsv'
# val_csv = f'{base_path}/data/CochlScene/val_shuffled_tiny.tsv'

feat_path = f'{base_path}/data/CochlScene/features/logmel128_scaled/'

experiments = 'cochlscene_test/'

if not os.path.exists(experiments):
    os.makedirs(experiments)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/work/user_data/jtaylor/home/.conda/envs/d20-keras/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [86]:
#train_aug_csv = generate_train_aug_csv(train_csv, aug_csv, feat_path, aug_path, experiments)
train_aug_csv = train_csv

num_audio_channels = 1
num_freq_bin = 128
num_classes = 13
max_lr = 0.1
batch_size = 64
num_epochs = 250
mixup_alpha = 0.4
crop_length = 400
sample_num = len(open(train_aug_csv, 'r').readlines()) - 1
print(f'Numbers of training samples: {sample_num}')


# compute delta and delta delta for validation data
data_val, y_val = load_data_2020(feat_path, val_csv, num_freq_bin, 'logmel')
print(f'{len(y_val)} validation samples with values {set(y_val)}')
print(f'{collections.Counter(y_val)}\n')
print(f'validation data shape: {data_val.shape}')


data_deltas_val = deltas(data_val)
print(f'validation data deltas shape: {data_deltas_val.shape}')

data_deltas_deltas_val = deltas(data_deltas_val)
print(f'validation data delta deltas shape: {data_deltas_val.shape}')

data_val = np.concatenate((data_val[:,:,4:-4,:],data_deltas_val[:,:,2:-2,:],data_deltas_deltas_val),axis=-1)
print(f'validation data concat shape: {data_val.shape}')

y_val = keras.utils.to_categorical(y_val, num_classes)

Numbers of training samples: 19999
4999 validation samples with values {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Counter({11: 403, 3: 396, 2: 394, 0: 393, 4: 392, 1: 390, 8: 385, 5: 381, 6: 379, 12: 377, 10: 375, 9: 369, 7: 365})

validation data shape: (4999, 128, 431, 1)
validation data deltas shape: (4999, 128, 427, 1)
validation data delta deltas shape: (4999, 128, 427, 1)
validation data concat shape: (4999, 128, 423, 3)


In [87]:
model = model_fcnn(num_classes, input_shape=[num_freq_bin, None, 3*num_audio_channels], num_filters=[48, 96, 192], wd=0)

In [90]:
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer =SGD(lr=max_lr,decay=1e-6, momentum=0.9, nesterov=False),
              metrics=['accuracy'])

model.summary()

lr_scheduler = LR_WarmRestart(nbatch=np.ceil(sample_num/batch_size), Tmult=2,
                              initial_lr=max_lr, min_lr=max_lr*1e-4,
                              epochs_restart = [3.0, 7.0, 15.0, 31.0, 63.0,127.0]) 
save_path = experiments + "/model-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(save_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks = [lr_scheduler, checkpoint]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 128, None, 3) 0                                            
__________________________________________________________________________________________________
batch_normalization_89 (BatchNo (None, 128, None, 3) 12          input_9[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_65 (ZeroPadding2 (None, 132, None, 3) 0           batch_normalization_89[0][0]     
__________________________________________________________________________________________________
conv2d_73 (Conv2D)              (None, 64, None, 144 10800       zero_padding2d_65[0][0]          
__________________________________________________________________________________________________
batch_norm

In [91]:
# Due to the memory limitation, in the training stage we split the training data
train_data_generator = Generator_timefreqmask_withdelta_splitted(feat_path, train_aug_csv, num_freq_bin,
                              batch_size=batch_size,
                              alpha=mixup_alpha,
                              crop_length=crop_length, splitted_num=5, classes=num_classes)()

history = model.fit_generator(train_data_generator,
                              validation_data=(data_val, y_val),
                              epochs=num_epochs, 
                              verbose=1, 
                              workers=4,
                              max_queue_size = 100,
                              callbacks=callbacks,
                              steps_per_epoch=np.ceil(sample_num/batch_size)
                              ) 

Epoch 1/250
   2/2000 [..............................] - ETA: 2:24:14 - loss: 2.5626 - acc: 0.1000

/work/user_data/jtaylor/home/.conda/envs/d20-keras/lib/python3.6/site-packages/keras/callbacks.py:99: UserWarning: Method on_batch_begin() is slow compared to the batch update (0.982896). Check your callbacks.
  % delta_t_median)


 294/2000 [===>..........................] - ETA: 4:47 - loss: 2.5652 - acc: 0.0738